In [2]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [4]:
# Load Data
CSV_PATH = "Resaleflatprices.csv"  
df = pd.read_csv(CSV_PATH)

print("Shape:", df.shape)
df.head(10)

Shape: (216946, 11)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
5,2017-01,ANG MO KIO,3 ROOM,150,ANG MO KIO AVE 5,01 TO 03,68.0,New Generation,1981,63 years,275000.0
6,2017-01,ANG MO KIO,3 ROOM,447,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1979,61 years 06 months,280000.0
7,2017-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,04 TO 06,67.0,New Generation,1976,58 years 04 months,285000.0
8,2017-01,ANG MO KIO,3 ROOM,447,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1979,61 years 06 months,285000.0
9,2017-01,ANG MO KIO,3 ROOM,571,ANG MO KIO AVE 3,01 TO 03,67.0,New Generation,1979,61 years 04 months,285000.0


In [6]:
# Dtypes 
print("Dtypes:\n", df.dtypes, "\n")

# Missing values
print("Missing values per column:\n", df.isna().sum(), "\n")

# Unique counts 
print("Unique counts per column:\n", df.nunique())

Dtypes:
 month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
dtype: object 

Missing values per column:
 month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64 

Unique counts per column:
 month                   105
town                     26
flat_type                 7
block                  2743
street_name             576
storey_range             17
floor_area_sqm          186
flat_model               21
lease_commence_date      57
remaining_lease         692
resale_pr

In [8]:
# Convert the 'month' string (e.g., "2019-03") into a pandas datetime object
df["month"] = pd.to_datetime(df["month"], format="%Y-%m")

# Extract the calendar year (int) from the datetime column 
df["year"] = df["month"].dt.year

# Extract the numeric month (1–12) from the datetime column 
df["month_num"] = df["month"].dt.month

# Check of the new columns created
df[["month", "year", "month_num"]].sample(5, random_state=42)

,month,year,month_num
45669,2019-03-01,2019,3
82571,2020-10-01,2020,10
62664,2019-12-01,2019,12
155290,2023-12-01,2023,12
179062,2024-08-01,2024,8


In [10]:
# Convert storey_range to storey_mid 
def parse_storey(s):
    """
    Convert a storey range like '10 TO 12' into its numeric midpoint (e.g., 11.0).
    """
    try:
        low, high = s.split(" TO ")           # Split by the literal string " TO "
        low_i, high_i = int(low), int(high)   # Convert the two tokens to integers
        return (low_i + high_i) / 2.0         # Midpoint of the range
    except Exception:
        return np.nan                          # Fallback for any bad/missing values

# Apply the parser to the whole column to create a numeric feature.
df["storey_mid"] = df["storey_range"].apply(parse_storey)

# Check
df[["storey_range", "storey_mid"]].sample(5, random_state=42)

,storey_range,storey_mid
45669,01 TO 03,2.0
82571,01 TO 03,2.0
62664,07 TO 09,8.0
155290,01 TO 03,2.0
179062,01 TO 03,2.0


In [12]:
# Convert remaining_lease to remaining_lease_years 
def parse_remaining_lease(s):
    """
    Parse text like '61 years 04 months' or '77 years' and return decimal years.
    Examples:
      '61 years 04 months' -> 61 + 4/12 = 61.333...
      '77 years'           -> 77.0
    """
    try:
        s = s.strip()                          # Remove surrounding whitespace
        parts = s.split()                      # Tokenize by whitespace
        years = int(parts[0])                  # First token should be the integer years

        months = 0
        # Look for a numeric token that represents months 
        # Typical positions: parts[2] is the month number when present.
        # Cannot use months = int(parts[2]) because NaN if remaining_lease have no months
        for tok in parts[1:]:
            # If we find a purely digit token after 'years', treat it as months
            if tok.isdigit():
                months = int(tok)
                break

        return years + months / 12.0
    except Exception:
        return np.nan

# Apply parser to the text column to produce a numeric feature in years.
df["remaining_lease_years"] = df["remaining_lease"].apply(parse_remaining_lease)

# Check
df[["remaining_lease", "remaining_lease_years"]].sample(5, random_state=42)

,remaining_lease,remaining_lease_years
45669,64 years,64.000000
82571,77 years 10 months,77.833333
62664,50 years 07 months,50.583333
155290,70 years 10 months,70.833333
179062,60 years 11 months,60.916667


In [14]:
# Flat_age (from remaining lease) 
# Every HDB flat starts with a 99-year lease.

df["flat_age"] = 99 - df["remaining_lease_years"]

# Check: compare remaining lease vs calculated flat_age
df[["remaining_lease", "remaining_lease_years", "flat_age"]].sample(5, random_state=42)


,remaining_lease,remaining_lease_years,flat_age
45669,64 years,64.000000,35.000000
82571,77 years 10 months,77.833333,21.166667
62664,50 years 07 months,50.583333,48.416667
155290,70 years 10 months,70.833333,28.166667
179062,60 years 11 months,60.916667,38.083333


In [16]:
# Save the cleaned/preprocessed dataset
PROCESSED_PATH = "Resaleflatprices_preprocessed.csv"
df.to_csv(PROCESSED_PATH, index=False)

print(f"Processed dataset saved to: {PROCESSED_PATH}")

Processed dataset saved to: Resaleflatprices_preprocessed.csv
